In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")
print("Train shape : ", df_train.shape)
print("Test shape : ", df_test.shape)
cata_cols = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

Train shape :  (4209, 378)
Test shape :  (4209, 377)


In [2]:
alldata = pd.concat([df_train, df_test], sort=False)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata[cata_cols] = alldata[cata_cols].apply(le.fit_transform)
df_train = alldata.iloc[:len(df_train), :]
df_test = alldata.iloc[len(df_train):, :]

y = df_train['y']
X = df_train.drop(['y'], axis=1)
X_test = df_test.drop(['y'], axis=1)

In [3]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [4]:
kf = KFold(n_splits=5, shuffle=True, random_state=41)
models = []
best_iter = 0
_y_predict = np.zeros(X.shape[0])

In [5]:
for train_index, valid_index in kf.split(X):
    X_train, X_valid = X.iloc[train_index, :], X.iloc[valid_index, :]
    y_train, y_valid = y.values[train_index], y.values[valid_index]
    model = xgb.XGBRegressor(learning_rate=0.01, max_depth=3, colsample_bytree=0.45, n_estimators=10000, seed=1,)
    model.fit(X_train, y_train,  early_stopping_rounds=75, verbose=False, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric="rmse")
    best_iter += model.best_iteration
    _y_predict[valid_index] = model.predict(X_valid)
    #models.append(model)

In [6]:
best_iter

3481

In [7]:
_y_predict

array([100.48852539, 102.81568146,  88.02734375, ..., 110.09768677,
        93.51220703,  92.59830475])

In [8]:
best_iter //= 5
model = xgb.XGBRegressor(learning_rate=0.01, max_depth=3, colsample_bytree=0.45, n_estimators=best_iter,seed=1,)
model.fit(X, y)

#r2 = r2_score(y, model.predict(xgb.DMatrix(X)))
#print('R2-train = %.4f' % r2)

_y_test = model.predict(X_test)
submit = pd.DataFrame({'ID': df_test['ID'], 'y': _y_test})
submit.to_csv('benz-xgb.csv', index=False)